In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.datasets import load_boston
from sklearn.metrics import r2_score, mean_squared_log_error, mean_squared_error,mean_absolute_error
import datetime as dt
import pickle
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import preparing_data as F

In [2]:
data=pd.read_pickle("./dataframe/df_20210912_133401_ONLY_RISKY_EVENTS-6-withFE.pkl")
data.reset_index(inplace=True)
data.drop(['index'], inplace=True, axis=1)
print(data.shape)
data.head()

(272, 86)


,__time_to_tca,MISS_DISTANCE,RELATIVE_SPEED,RELATIVE_POSITION_R,RELATIVE_POSITION_T,RELATIVE_POSITION_N,RELATIVE_VELOCITY_R,RELATIVE_VELOCITY_T,RELATIVE_VELOCITY_N,COLLISSION_PROBABILITY,...,PC_mavg_1,PC_trend_1,PC_trend_3,PC_gradient_1,PC_gradient_3,MD_mavg_1,MD_trend_1,MD_trend_3,MD_gradient_1,MD_gradient_3
0,0.832257,2244.0,11172.0,87.9,-1474.9,1689.9,-201.4,-8424.3,-7336.0,-3.418392,...,-3.475175,0.132439,0.590914,0.354472,0.579354,2181.333333,47.0,17.0,125.795356,16.667421
1,0.580855,966.0,212.0,191.0,-944.8,-68.9,33.8,-8.4,209.3,-3.711974,...,-3.981494,0.310256,0.575376,0.747481,0.536846,1131.000000,-201.0,-326.0,-484.256505,-304.169574
2,0.189825,1102.0,212.0,193.3,-1083.1,-75.0,33.9,-8.4,209.3,-3.913996,...,-3.882734,-0.202022,0.296282,-0.516640,0.270366,1078.333333,136.0,-158.0,347.799165,-144.179478
3,0.994637,456.0,11707.0,-22.1,288.8,353.3,-10.2,-9077.7,7392.6,-2.055073,...,-2.841298,1.280567,1.312874,3.634346,1.445698,709.666667,-577.0,124.0,-1637.570088,136.545083
4,0.759012,629.0,11707.0,-18.7,395.3,489.9,-10.3,-9077.7,7392.6,-2.649558,...,-2.680091,-0.594485,0.483623,-2.523011,0.486607,706.000000,173.0,-11.0,734.217036,-11.067877


In [3]:
#data=data.head(100)

In [4]:
data.shape

(272, 86)

In [5]:
# ############################### FEATURE ENGINEERING ##########################################
# # Gradient: Miss distance two last CDM
# data["_GRADIENT_MISS_DISTANCE_34"]=(-data.MISS_DISTANCE_3+data.MISS_DISTANCE_4)/abs(data.__time_to_tca_4-data.__time_to_tca_3)
# # Gradient: Miss distance first and last CDM
# data["_GRADIENT_MISS_DISTANCE_14"]=(-data.MISS_DISTANCE+data.MISS_DISTANCE_4)/abs(data.__time_to_tca_4-data.__time_to_tca)
# #Gradient: COLLISSION PROBABILITY two last CDM
# data["_GRADIENT_PC_34"]=(-data.COLLISSION_PROBABILITY_3+data.COLLISSION_PROBABILITY_4)/abs(data.__time_to_tca_4-data.__time_to_tca_3)
# #Gradient: COLLISSION PROBABILITY first and last CDM
# data["_GRADIENT_PC_14"]=(-data.COLLISSION_PROBABILITY+data.COLLISSION_PROBABILITY_4)/abs(data.__time_to_tca_4-data.__time_to_tca)

In [6]:
data.shape

(272, 86)

In [7]:
#print(list(data.columns))

In [8]:
# aux=data
# data=data.loc[:, data.columns.str.endswith('4','TARGET')]
# data["COLLISSION_PROBABILITY_TARGET"]=aux.COLLISSION_PROBABILITY_TARGET
# print(list(data.columns))
# data.shape

In [9]:
train, test = train_test_split(data, test_size=0.30, random_state=42)

In [10]:
print("Train dataframe dimension {} x {}".format(train.shape[0],train.shape[1]))
print("Test dataframe dimension {} x {}".format(test.shape[0],test.shape[1]))

Train dataframe dimension 190 x 86
Test dataframe dimension 82 x 86


In [11]:
# Y_train = train["COLLISSION_PROBABILITY_TARGET"]
# X_train= train.drop(["COLLISSION_PROBABILITY_TARGET"], axis=1)
# Y_test = test["COLLISSION_PROBABILITY_TARGET"]
# X_test= test.drop(["COLLISSION_PROBABILITY_TARGET"], axis=1)


In [12]:
Y_train = train["COLLISSION_PROBABILITY"]
X_train= train.drop(["COLLISSION_PROBABILITY"], axis=1)
Y_test = test["COLLISSION_PROBABILITY"]
X_test= test.drop(["COLLISSION_PROBABILITY"], axis=1)


In [13]:
X = X_train
y = Y_train

In [14]:
def bayesian_opt_lgbm(X, y, init_iter=3, n_iters=7, random_state=11, seed = 101, num_iterations = 100):
      dtrain = lgb.Dataset(data=X, label=y)
      #Metric evaluation functions
      def lgb_r2_score(preds, dtrain):                #R2
            labels = dtrain.get_label()
            return 'metric', r2_score(labels, preds), True
      def lgb_root_squared_error(preds, dtrain):      #RMSE
            labels = dtrain.get_label()
            return 'metric', mean_squared_error(labels, preds,squared=False), True
      def lgb_mean_absolute_error(preds, dtrain):     #MAE
            labels = dtrain.get_label()
            return 'metric', mean_absolute_error(labels, preds), True
      def lgb_adjusted_r2_score(preds, dtrain):       #ADJUSTED R2
            labels = dtrain.get_label()
            n=dtrain.num_data()
            k=dtrain.num_feature()
            return 'metric', ((1-r2_score(labels, preds))*(n-1))/(n-k-1), True
            
      # Select metric
      metric='lgb_r2_score'
      metric_feval=lgb_r2_score

      # Objective Function
      def hyp_lgbm(num_leaves, feature_fraction, learning_rate, bagging_fraction, max_depth, min_split_gain, min_child_weight):
              params = {      'application':'regression',
                              'num_iterations': num_iterations,
                              'early_stopping_round':50,
                              'verbose':-1,
                              'metric':metric} # Default parameters
              params["num_leaves"] = int(round(num_leaves))
              params["learning_rate"] = learning_rate
              params['feature_fraction'] = max(min(feature_fraction, 1), 0)
              params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
              params['max_depth'] = int(round(max_depth))
              params['min_split_gain'] = min_split_gain
              params['min_child_weight'] = min_child_weight
              cv_results = lgb.cv(params, dtrain, nfold=5, seed=seed,categorical_feature=[], stratified=False,
                                  verbose_eval =None, feval=metric_feval)
              #print(cv_results)
              return np.max(cv_results['metric-mean'])
    
              # Domain space-- Range of hyperparameters 
      pds = {     'num_leaves': (60, 120),
                  'feature_fraction': (0.1, 0.9),
                  'bagging_fraction': (0.7, 1),
                  'max_depth': (7, 15),
                  'learning_rate':(0.001,0.05), 
                  'min_split_gain': (0.001, 0.1),
                  'min_child_weight': (10, 35)
                  }
      # Surrogate model
      optimizer = BayesianOptimization(hyp_lgbm, pds, random_state=random_state)
                                          
      # Optimize
      optimizer.maximize(init_points=init_iter, n_iter=n_iters)

      filename="./opt_parameters_bo/opt_parameters_df_singles_{}_{}_RISKY_EVENTS-6_FE.pkl".format(dt.datetime.now().strftime("%Y%m%d_%H%M%S"),metric)
      a_file = open(filename, "wb")
      pickle.dump(optimizer.max['params'], a_file)
      a_file.close()

      return optimizer

bayesian_ouput=bayesian_opt_lgbm(X, y, init_iter=5, n_iters=500, random_state=77, seed = 101,num_iterations=300)

|   iter    |  target   | baggin... | featur... | learni... | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.848    |  0.9757   |  0.6138   |  0.03793  |  8.115    |  12.18    |  0.07901  |  79.57    |
|  2        |  0.8      |  0.8623   |  0.2922   |  0.02773  |  10.2     |  27.88    |  0.08383  |  95.31    |
|  3        |  0.8222   |  0.7888   |  0.3248   |  0.03557  |  10.38    |  11.43    |  0.07496  |  87.14    |
|  4        |  0.5445   |  0.7527   |  0.1395   |  0.01533  |  7.534    |  28.78    |  0.007313 |  85.91    |
|  5        |  0.7843   |  0.8093   |  0.2216   |  0.02779  |  10.55    |  10.9     |  0.08247  |  76.4     |
|  6        |  0.8548   |  1.0      |  0.9      |  0.05     |  7.0      |  10.0     |  0.1      |  120.0    |
|  7        |  0.8325   |  1.0      |  0.9      |  0.05     |  15.0     |  35.0     |  0.001    |  120.0    |
|  8      

In [15]:
opt_parameters=bayesian_ouput.max['params']
opt_parameters

{'bagging_fraction': 0.8693796326739299,
 'feature_fraction': 0.8168553631386558,
 'learning_rate': 0.0464041915708064,
 'max_depth': 13.009239099967989,
 'min_child_weight': 10.19191247052637,
 'min_split_gain': 0.004793459623540061,
 'num_leaves': 93.8608845306346}

In [16]:
# filename="opt_parameters_balanced_df_adjusted_{}_corr_mae_RISKY_EVENTS-8.pkl".format(dt.datetime.now().strftime("%Y%m%d_%H%M%S"))
# a_file = open(filename, "wb")

# pickle.dump(opt_parameters, a_file)

# a_file.close()

In [17]:
# filename="./opt_parameters_bo/opt_parameters_balanced_df_adjusted_20210911_215733_corr_lgb_r2_score_RISKY_EVENTS-8-lastcdm-BEST-MODEL.pkl"
# a_file = open(filename,"rb")
# output = pickle.load(a_file)
# opt_parameters=output
# output

In [18]:
#optimizer.max['params']

In [19]:
#'bagging_fraction': 1.0, 'feature_fraction': 0.9, 'max_depth': 8.0, 'min_child_weight': 25.0, 'min_split_gain': 0.013771321931506838, 'num_leaves': 88.93816438820497}

In [20]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'regression_l2',
    'learning_rate': opt_parameters.get("learning_rate"),
    'feature_fraction': opt_parameters.get("feature_fraction"),
    'bagging_fraction': opt_parameters.get("bagging_fraction"),
    #'bagging_freq': 10,
    'verbose': -1,
    "max_depth": int(round(opt_parameters.get("max_depth"))),
    "num_leaves": int(round(opt_parameters.get("num_leaves"))),  
    #"max_bin": 512,
    'min_split_gain' : opt_parameters.get("min_split_gain"),
    "num_iterations": 300,
    "n_estimators": 500,
    'min_child_weight' : opt_parameters.get("min_child_weight")
}

In [21]:
#Construct a gradient boosting model.
#gbm = lgb.LGBMRegressor(**hyper_params)

In [22]:
lgbm_train = lgb.Dataset(X, label=y)
#lgbm_eval = lgb.Dataset(X_test, label=Y_test,reference=lgbm_train)

In [23]:
gbm = lgb.train(params=hyper_params,
                train_set=lgbm_train,
                #valid_sets=lgbm_eval,
                #verbose_eval=20,
                #eval_metric='lgb_r2_score',
                #early_stopping_rounds=100
                )

In [24]:
#Build a gradient boosting model from the training set (X, y)
""" gbm.fit(X, y,
        eval_set=[(X_test, Y_test)],
        eval_metric='l1',
        early_stopping_rounds=50) """


" gbm.fit(X, y,\n        eval_set=[(X_test, Y_test)],\n        eval_metric='l1',\n        early_stopping_rounds=50) "

In [25]:
Y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

In [26]:
# print('The r2 of prediction is:', r2_score(y, Y_pred))
# print('The MSE of prediction is:', mean_squared_error(y, Y_pred, squared=True))
# print('The RMSE of prediction is:', mean_squared_error(y, Y_pred, squared=False))

In [27]:
print('The r2 of prediction is:', r2_score(Y_test, Y_pred))
print('The MSE of prediction is:', mean_squared_error(Y_test, Y_pred, squared=True))
print('The RMSE of prediction is:', mean_squared_error(Y_test, Y_pred, squared=False))
print('The MAE of prediction is:', mean_absolute_error(Y_test, Y_pred))

The r2 of prediction is: 0.9068312762554236
The MSE of prediction is: 0.05145180527554441
The RMSE of prediction is: 0.22682990383885546
The MAE of prediction is: 0.15057238630085704


In [28]:
# df_train = lgb.Dataset(data=X_test, label=Y_test)
# def lgb_adjusted_r2_score(preds, dtrain):
#     labels = dtrain.get_label()
#     n=dtrain.num_data()
#     k=dtrain.num_feature()
#     return 'metric', ((1-r2_score(labels, preds))*(n-1))/(n-k-1), True

In [29]:
#lgb_adjusted_r2_score(Y_pred, df_train)

### Validating model

In [30]:
aux_y=pd.DataFrame(Y_test)
aux_y.reset_index(inplace=True)
aux_y.drop(['index'], inplace=True, axis=1)
aux_y_pred=pd.DataFrame(Y_pred)
aux_y_pred.reset_index(inplace=True)
aux_y_pred.drop(['index'], inplace=True, axis=1)
frames=[aux_y,aux_y_pred]
result=pd.concat(frames,axis=1)
result.columns=["y_true","y_predicted"]
result["y_true_10"]=10**result.y_true
result["y_predicted_10"]=10**result.y_predicted
result

,y_true,y_predicted,y_true_10,y_predicted_10
0,-4.558148,-4.619382,0.000028,0.000024
1,-5.849858,-5.866170,0.000001,0.000001
2,-4.882729,-5.140053,0.000013,0.000007
3,-5.337053,-5.338003,0.000005,0.000005
4,-5.325139,-5.360476,0.000005,0.000004
...,...,...,...,...
77,-5.017593,-4.997042,0.000010,0.000010
78,-5.578232,-5.503793,0.000003,0.000003
79,-4.345823,-4.258162,0.000045,0.000055
80,-5.901702,-5.718289,0.000001,0.000002


In [31]:
result[result["y_true_10"]>0.00001]

,y_true,y_predicted,y_true_10,y_predicted_10
0,-4.558148,-4.619382,0.000028,0.000024
2,-4.882729,-5.140053,0.000013,0.000007
7,-4.982967,-5.188391,0.000010,0.000006
13,-4.387110,-4.214612,0.000041,0.000061
19,-4.407934,-4.524030,0.000039,0.000030
22,-4.201418,-4.322833,0.000063,0.000048
23,-4.775985,-4.719754,0.000017,0.000019
25,-3.254067,-3.676642,0.000557,0.000211
26,-2.528122,-3.618370,0.002964,0.000241
29,-4.119529,-4.047112,0.000076,0.000090


In [32]:
result[result["y_true_10"]>0.0001]

,y_true,y_predicted,y_true_10,y_predicted_10
25,-3.254067,-3.676642,0.000557,0.000211
26,-2.528122,-3.618370,0.002964,0.000241
35,-3.716021,-3.688654,0.000192,0.000205
43,-3.501965,-3.679102,0.000315,0.000209
50,-3.753994,-3.686418,0.000176,0.000206
60,-3.513428,-3.685072,0.000307,0.000207
63,-3.812197,-4.150124,0.000154,0.000071
70,-3.499215,-3.583688,0.000317,0.000261
